In [ ]:
"""
XSA Python buildpack app example
Author: Andrew Lunde
"""

Import other python libs

In [ ]:
import os
import json


Import the Cloud Foundry Environment library.

This makes it easier to get info from the application's environment.

https://pypi.org/project/cfenv/ for details.


In [ ]:
from cfenv import AppEnv


Load the HANA Python Client library

In [ ]:
from hdbcli import dbapi


Create the Flask webserver application object

Grab the environment

In [ ]:
env = AppEnv()


```
Get port from environment variable or choose 9099 as local default
If you are testing locally (i.e. not with xs or cf deployments,
Be sure to pull all the python modules locally
with pip using XS_PYTHON unzipped to /tmp
pip install -r requirements.txt
```


NodeJS way xsenv.loadEnv();


In [ ]:
with open('default-env.json') as json_file:
    data = json.load(json_file)
    print(data['PORT'])
    os.environ['PORT'] = str(data['PORT'])
    #print(json.dumps(data['VCAP_SERVICES']))
    os.environ['VCAP_SERVICES'] = json.dumps(data['VCAP_SERVICES'])
    


In [ ]:
port = int(os.getenv("PORT", 9099))
print(port)

In [ ]:
hana = env.get_service(label='hana')
print(hana)

First insert a key/value pair.

In [ ]:
output = 'Python UnAuthorized SecureStore Insert. \n'


In [ ]:
schema = hana.credentials['schema']
host = hana.credentials['host']
port = hana.credentials['port']
user = hana.credentials['user']
password = hana.credentials['password']


In [ ]:
if 'certificate' in hana.credentials:
    haascert = hana.credentials['certificate']


In [ ]:
print("schema: " + schema)
print("host: " + host)
print("port: " + port)
print("user: " + user)
print("password: " + password)

In [ ]:
output += 'schema: ' + schema + '\n'
output += 'host: ' + host + '\n'
output += 'port: ' + port + '\n'
output += 'user: ' + user + '\n'
output += 'pass: ' + password + '\n'


In [ ]:
# Connect to the python HANA DB driver using the connection info
# User for HANA as a Service instances
if 'certificate' in hana.credentials:
    connection = dbapi.connect(
        address=host,
        port=int(port),
        user=user,
        password=password,
        currentSchema=schema,
        encrypt="true",
        sslValidateCertificate="true",
        sslCryptoProvider="openssl",
        sslTrustStore=haascert
    )
else:
    connection = dbapi.connect(
        address=host,
        port=int(port),
        user=user,
        password=password,
        currentSchema=schema
    )



In [ ]:
# Prep a cursor for SQL execution
cursor = connection.cursor()


In [ ]:
# Form an SQL statement to retrieve some data

string2store = 'Whatever!'


In [ ]:
import codecs


In [ ]:
hex2store = (codecs.encode(str.encode(string2store), "hex")).decode()


In [ ]:

try:
    cursor.callproc("SYS.USER_SECURESTORE_INSERT",
                    ("TestStoreName", False, "TestKey", hex2store))
    output += 'key TestKey with value ' + string2store + '=' + \
                hex2store + ' was inserted into store TestStoreName.\n'
except:
    output += 'key TestKey likely already exists. Try deleting first.\n'


In [ ]:
# Close the DB connection
connection.close()


In [ ]:
print(output)

Now read it back from the secure store.

In [ ]:
output = 'Python UnAuthorized SecureStore Retrieve.\n'

In [ ]:
schema = hana.credentials['schema']
host = hana.credentials['host']
port = hana.credentials['port']
user = hana.credentials['user']
password = hana.credentials['password']

In [ ]:
if 'certificate' in hana.credentials:
    haascert = hana.credentials['certificate']

In [ ]:
output += 'schema: ' + schema + '\n'
output += 'host: ' + host + '\n'
output += 'port: ' + port + '\n'
output += 'user: ' + user + '\n'
output += 'pass: ' + password + '\n'


In [ ]:
# Connect to the python HANA DB driver using the connection info
# User for HANA as a Service instances
if 'certificate' in hana.credentials:
    connection = dbapi.connect(
        address=host,
        port=int(port),
        user=user,
        password=password,
        currentSchema=schema,
        encrypt="true",
        sslValidateCertificate="true",
        sslCryptoProvider="openssl",
        sslTrustStore=haascert
    )
else:
    connection = dbapi.connect(
        address=host,
        port=int(port),
        user=user,
        password=password,
        currentSchema=schema
    )

In [ ]:
# Prep a cursor for SQL execution
cursor = connection.cursor()

In [ ]:
# Form an SQL statement to retrieve some data

# https://blogs.sap.com/2017/07/26/sap-hana-2.0-sps02-new-feature-updated-python-driver/

# cursor.execute('call SYS.USER_SECURESTORE_RETRIEVE (\'TestStoreName\',
# False, \'TestKey\', ?)')
hexvalue = cursor.callproc("SYS.USER_SECURESTORE_RETRIEVE",
                            ("TestStoreName", False, "TestKey", None))


In [ ]:
# Close the DB connection
connection.close()


In [ ]:
import codecs


In [ ]:
  
if hexvalue[3] is None:
    output += 'key TestKey does not exist in store TestStoreName. ' + \
                'Try inserting a value first.\n'
else:
    retrieved = codecs.decode(hexvalue[3].hex(), "hex").decode()
    output += 'key TestKey with value ' + retrieved + \
                ' was retrieved from store TestStoreName.\n'


In [ ]:
print(output)

In [ ]:
output = 'Python UnAuthorized SecureStore Delete. \n'


In [ ]:
schema = hana.credentials['schema']
host = hana.credentials['host']
port = hana.credentials['port']
user = hana.credentials['user']
password = hana.credentials['password']


In [ ]:
# The certificate will available for HANA service instances
# that require an encrypted connection
# Note: This was tested to work with python hdbcli-2.3.112 tar.gz package
# not hdbcli-2.3.14 provided in XS_PYTHON00_0-70003433.ZIP
if 'certificate' in hana.credentials:
    haascert = hana.credentials['certificate']


In [ ]:

output += 'schema: ' + schema + '\n'
output += 'host: ' + host + '\n'
output += 'port: ' + port + '\n'
output += 'user: ' + user + '\n'
output += 'pass: ' + password + '\n'


In [ ]:
# Connect to the python HANA DB driver using the connection info
# User for HANA as a Service instances
if 'certificate' in hana.credentials:
    connection = dbapi.connect(
        address=host,
        port=int(port),
        user=user,
        password=password,
        currentSchema=schema,
        encrypt="true",
        sslValidateCertificate="true",
        sslCryptoProvider="openssl",
        sslTrustStore=haascert
    )
else:
    connection = dbapi.connect(
        address=host,
        port=int(port),
        user=user,
        password=password,
        currentSchema=schema
    )


In [ ]:
# Prep a cursor for SQL execution
cursor = connection.cursor()


In [ ]:
# Form an SQL statement
cursor.callproc("SYS.USER_SECURESTORE_DELETE",
                ("TestStoreName", False, "TestKey"))


In [ ]:
# Close the DB connection
connection.close()


In [ ]:
output += 'key TestKey was deleted from store TestStoreName.\n'


In [ ]:
print(output)